In [2]:
## 并行方式，
import pandas as pd
import numpy as np
import torch as th
from tqdm import tqdm
from multiprocessing import Pool

import os
import pickle
import dgl

graphs, _ = dgl.load_graphs(os.path.join('../dataset/', 'graph.bin'))
def minMaxScaling(data):
    min, max = data.min(), data.max()
    return min + (data - min)/(max - min)
    
edges = th.cat((minMaxScaling(graphs[0].in_degrees().unsqueeze_(1).float().add(1).log()), minMaxScaling(graphs[0].out_degrees().unsqueeze_(1).float().add(1).log())), dim=1)
# edges

# test_num = 2000
test_num = graphs[0].num_nodes()
# workers = multiprocessing.cpu_count() / 5
workers = 5

def feat_map(i):
    # return th.Tensor([[i]])
    return th.FloatTensor([
        edges[graphs[0].predecessors(i), 0].mean().item(), edges[graphs[0].predecessors(i), 0].std().item(),
        edges[graphs[0].predecessors(i), 1].mean().item(), edges[graphs[0].predecessors(i), 1].std().item(),
        edges[graphs[0].successors(i), 0].mean().item(), edges[graphs[0].successors(i), 0].std().item(),
        edges[graphs[0].successors(i), 1].mean().item(), edges[graphs[0].successors(i), 1].std().item(),
        ]).nan_to_num(0).unsqueeze(dim=0)

with Pool(workers) as p:
    rets = p.map(feat_map, range(test_num))

features_neigh = th.cat(rets, dim=0)
features_neigh = features_neigh.numpy()
print(features_neigh.shape)

with open(os.path.join('../dataset/', './features_neigh.npy'), 'wb') as f:
    np.save(f, features_neigh)

KeyboardInterrupt: 